<a href="https://colab.research.google.com/github/calerovillalobos10/Intro_Limpieza_Transf_Data/blob/main/Lab2_PF3347.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2 Limpieza y Transformación de Datos

## Estudiantes

*   Bryan Thomas Calero Villalobos
*   Daniela Montero Parkinson
*   Christopher Zúñiga Cárdenas

# Uso práctico

## Extracción de datos

Se ingresa a la página de [OpenQA](https://openaq.org/) para consumir el API que nos da la documentación, en este caso se utilizará para el trabajo el API V3. En la documentación del api se indica registrarse en [Registro](https://explore.openaq.org/register) e ir al apartado de *cuenta*, con el fin de capturar el API KEY necesario para consumir el API.

Como primer paso se instalan las librerías necesarias para poder consumir el API mencionado y capturar las variables de entorno

In [1]:
# Instalar las librerías necesarias
!pip3 install requests
!pip3 install python-dotenv

Se importan las librerías para poder utilizarlas en los métodos definidos

In [2]:
import requests
import os
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from urllib.parse import urlencode
from datetime import datetime

Se descargar el archivo del .env que se encuentra en drive

In [3]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1JrTQmCwUWuru9BJFCwuG25_aIT1uN4XR' -O .env

--2024-09-16 01:19:57--  https://drive.google.com/uc?export=download&id=1JrTQmCwUWuru9BJFCwuG25_aIT1uN4XR
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.101, 142.251.2.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1JrTQmCwUWuru9BJFCwuG25_aIT1uN4XR&export=download [following]
--2024-09-16 01:19:57--  https://drive.usercontent.google.com/download?id=1JrTQmCwUWuru9BJFCwuG25_aIT1uN4XR&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79 [application/octet-stream]
Saving to: ‘.env’

.env                100%[===================>]      79  --.-KB/s    in 0s      

2024-09

Obtener el api key para consumir la información del sitio Open AQ

In [48]:
load_dotenv('.env') # Cargar el archivo .env descargado
API_KEY = os.getenv('KEY_OpenAQ') # Obtener el API KEY de la variable de entorno

Se inicializa una variable con la url a consumir

In [49]:
api_url = f'https://api.openaq.org/v2/measurements'

Se definen los parámetros que se enviarán al enpoint y mediante este método se construye la url final a consumir

In [50]:
# Construye la URL con los parámetros necesarios
def build_url(country, start_date, end_date, page, limit):
  # Parámetros para la solicitud: país, fechas y página actual
  params = {
    'date_from': start_date,
    'date_to': end_date,
    'limit': limit,  # Ajusta el límite según lo necesario
    'page': page,
    'country': country,
    'sort': 'desc',
    'order_by': 'datetime'
  }

  return f"{api_url}?{urlencode(params)}"

Este método se implementó para solucionar el problema Request Time-out que arroja el api, dado a la cantidad de información que trae. Se funcionalidad es crear una sesión HTTP que aplica reintentos automáticos cuando se producen errores temporales o de conexión en solicitudes GET

In [51]:
# Método para crear la estrategia de reintentos
def create_retry_session(max_retries):
    retry_strategy = Retry(
        total=max_retries,  # Número máximo de reintentos
        status_forcelist=[408, 500, 502, 503, 504],  # Errores a los que se aplicarán los reintentos
        allowed_methods=["GET"],  # Solo reintentar en solicitudes GET
        backoff_factor=1  # Tiempo de espera entre reintentos
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)

    # Crear una sesión de requests con el adaptador de reintentos
    http = requests.Session()
    http.mount("https://", adapter)

    return http  # Devolver la sesión con la estrategia de reintentos

El siguiente método consume el API utilizando requests, obteniendo los datos de los países que proporciona la web. Este retorna una lista de diccionarios sobre los países con el método .json() utilizado en la línea data = response.json()

In [52]:
# Obtiene los datos de cada país
def fetch_country_data(country, start_date, end_date, limit, max_retries=1):
  all_data = [] # Lista para almacenar los datos obtenidos de todas las páginas

  try:

    page = 1  # Se inicia en la primera página
    has_more_data = True  # Control para el bucle
    headers = {"accept": "application/json", "X-API-Key": API_KEY} # Encabezados de la solicitud, incluyendo la API Key

    # Usar el nuevo método para crear la sesión con reintentos
    http = create_retry_session(max_retries)

    # Bucle para hacer solicitudes mientras haya más datos disponibles
    while has_more_data:
      # Crear la URL con los parámetros del país, fechas, página y límite
      url = build_url(country, start_date, end_date, page, limit)

      response = http.get(url, headers=headers) # Realiza la solicitud HTTP al API
      response.raise_for_status() # Lanza un error si la respuesta no es exitosa

      data = response.json().get('results', [])
      all_data.extend(data)  # Añade los datos de la página actual a la lista general

      if len(data) < limit:
        print(f"{country} No más datos")
        has_more_data = False # Si no hay más datos, se finaliza el bucle
      else:
        page += 1  # Si hay más datos, avanzar a la siguiente página

  except requests.RequestException as e:
    print(f"{country} No más datos")

  return all_data

La siguiente función obtiene datos de varios países en un rango de fechas específico y con un límite de resultados por país. Recorre una lista de países, llama a la función fetch_country_data para cada uno, y si se obtienen datos, los almacena en una lista que finalmente retorna.

In [53]:
# Función para obtener los datos de varios países
def get_countries_data(countries, start_date, end_date, limit):

  all_data = [] # Lista para almacenar todos los datos obtenidos

  # Ciclo sobre la lista de países
  for country in countries:

    print(f"Obteniendo datos para {country}...")
    # Llamada a una función que obtiene los datos del país en un periodo específico
    data = fetch_country_data(country, start_date, end_date, limit)

    if any(data):
      all_data.extend(data) # si existen datos se añaden los resultados a la lista general

  return all_data

Parámetros para definir los datos que se quieren obtener del api de open qa

In [58]:
# Parámetros para el enpoint
countries = ['CR', 'CL', 'EC', 'PE', 'AR', 'CO']  # Costa Rica, Chile, Ecuador, Peru, Argentina, Colombia
start_date = '2014-01-01T00:00:00Z'
end_date = datetime.utcnow().isoformat() + "Z"

data = get_countries_data(countries, start_date, end_date, 15000)

Obteniendo datos para CR...
CR No más datos
Obteniendo datos para CL...
CL No más datos
Obteniendo datos para EC...
EC No más datos
Obteniendo datos para PE...
PE No más datos
Obteniendo datos para AR...
AR No más datos
Obteniendo datos para CO...
CO No más datos


In [59]:
#Info solo para visualizar, se borrará
import pandas as pd
from pandas import json_normalize

# Normalizar los datos
df_results = json_normalize(data)

# Si hay parámetros anidados dentro de los resultados, normalizarlos también
if 'parameters' in df_results.columns:
  df_parameters = json_normalize(df_results['parameters'].explode())
  df_results = df_results.drop(columns=['parameters']).join(df_parameters, rsuffix='_param')

# Mostrar el DataFrame
df_results

# Definir el nombre del archivo CSV
archivo_csv = 'datos_normalizados.csv'

# Convertir el DataFrame a un archivo CSV
df_results.to_csv(archivo_csv, index=False)

print(f'DataFrame guardado en {archivo_csv}')

# Mostrar el DataFrame
print(df_results)

DataFrame guardado en datos_normalizados.csv
        locationId                                         location parameter  \
0             7077                     US Diplomatic Post: San Jose      pm25   
1             7077                     US Diplomatic Post: San Jose      pm25   
2             7077                     US Diplomatic Post: San Jose      pm25   
3             7077                     US Diplomatic Post: San Jose      pm25   
4             7077                     US Diplomatic Post: San Jose      pm25   
...            ...                                              ...       ...   
486338        7063  MED-ARAN - Medellín, Aranjuez - I.E Ciro Mendia      pm25   
486339        7063  MED-ARAN - Medellín, Aranjuez - I.E Ciro Mendia      pm25   
486340        7063  MED-ARAN - Medellín, Aranjuez - I.E Ciro Mendia      pm25   
486341        7063  MED-ARAN - Medellín, Aranjuez - I.E Ciro Mendia      pm25   
486342        7063  MED-ARAN - Medellín, Aranjuez - I.E Ciro Men

## Limpieza de datos

## Transformación y Visualización de datos